In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import logging
import pandas as pd
import wandb
import torch
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.config.model_args import MultiLabelClassificationArgs


In [ ]:
def process_labels(train):
    train['labels'] = ''
    for index, row in train.iterrows():
       train['labels'][index] = [row['misogynous'], row['shaming'], row['stereotype'], row['objectification'], row['violence']]

    train_data = train[['Text Transcription','labels']]
    train_data = train_data.rename(columns={'Text Transcription': 'text'})
    print(train_data.head(5))
    return train_data

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/MAMI/train/TRAINING/training.csv'

'/content/drive/MyDrive/Colab Notebooks/MAMI/train/TRAINING/training.csv'


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MAMI/train/TRAINING/training.csv', sep='\t')
train_data = process_labels(train)
print(train_data.head(5))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


                                                text           labels
0                                      Milk Milk.zip  [0, 0, 0, 0, 0]
1  ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S...  [1, 0, 0, 0, 1]
2  BREAKING NEWS: Russia releases photo of DONALD...  [0, 0, 0, 0, 0]
3                       MAN SEEKING WOMAN Ignad 18 O  [0, 0, 0, 0, 0]
4  Me explaining the deep lore of. J.R.R. Tolkein...  [0, 0, 0, 0, 0]
                                                text           labels
0                                      Milk Milk.zip  [0, 0, 0, 0, 0]
1  ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S...  [1, 0, 0, 0, 1]
2  BREAKING NEWS: Russia releases photo of DONALD...  [0, 0, 0, 0, 0]
3                       MAN SEEKING WOMAN Ignad 18 O  [0, 0, 0, 0, 0]
4  Me explaining the deep lore of. J.R.R. Tolkein...  [0, 0, 0, 0, 0]


In [ ]:
model_args = MultiLabelClassificationArgs(num_train_epochs=1)
model_args.num_train_epochs = 20
model_args.threshold = 0.8
model_args.no_save = True
model_args.overwrite_output_dir=True

cuda_available = torch.cuda.is_available()

model = MultiLabelClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=5,
    args=model_args,
    use_cuda=cuda_available
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
# Train the model
model.train_model(train_data,
                      show_running_loss=True,
                      use_early_stopping=True
                 )

In [ ]:
# Get the test list
# print('--------')
test = pd.read_csv('../input/mamidata/MAMI_test/MAMA_test/Test.csv', sep='\t')
test_original = test
test = test.rename(columns={'Text Transcription': 'text'})
test = test['text']
print("test.head(5): ", test.head(5))

In [ ]:
predictions, raw_outputs = model.predict(list(test))

print(predictions)

In [ ]:
with open("answer_0130_multi_label_taskB.txt", 'w', encoding="utf-8") as f:
    for filename, label in zip(test_original["file_name"], predictions):
        print(filename, label)
        f.write(f"{filename}\t{label}\n")